## Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import re

import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, f1_score, classification_report, accuracy_score

## Loading the dataset

In [ ]:
df = pd.read_csv('../input/spam-text-message-classification/SPAM text message 20170820 - Data.csv')

In [ ]:
df.head(3)

In [ ]:
df.shape

<b>The dataset has 5572 records and 2 columns</b>

In [ ]:
df.info()

<b>Both columns are of object datatype</b>

In [ ]:
# Checking if any duplicate records are present

duplicate=df[df.duplicated()] 
duplicate

<b>There are 415 duplicate records in the dataset</b>

In [ ]:
# Removing duplicate records

df.drop_duplicates(inplace=True)

In [ ]:
# Again check if any duplicate records are left

duplicate = df[df.duplicated()] 
duplicate

<b>Hence, all duplicate records are removed.</b>

In [ ]:
df.describe(include='object')

In [ ]:
# Checking for null values

df.isnull().sum()

<b>The dataset doesn't have any missing values</b>

In [ ]:
# Visualizing the disribution of ham and spam messages

sns.countplot(df['Category'])

<b> It can be seen that the count of spam messages is much less than ham. </b>

## Text Preprocessing

In [ ]:
# Expanding contractions

# Dictionary of English Contractions
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}

# Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

# Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)

# Expanding Contractions in the reviews
df['Message'] = df['Message'].apply(lambda x:expand_contractions(x))

In [ ]:
# Converting text to lowercase

df['Message'] = df['Message'].apply(lambda x:x.lower())

In [ ]:
# Removing digits and words containing digits

df['Message'] = df['Message'].apply(lambda x: re.sub('\w*\d\w*','', x))

In [ ]:
# Removing punctuations

df['Message'] = df['Message'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

In [ ]:
# Removing extra spaces

df['Message']=df['Message'].apply(lambda x: re.sub(' +',' ',x))

In [ ]:
# Displaying the text after cleaning

for index,text in enumerate(df['Message'][0:3]):
    print('Review %d:\n'%(index+1), text)

In [ ]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    rev = [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text) if w not in stopwords.words('english')]
    rev = ' '.join(rev)
    return rev

df['Message'] = df.Message.apply(lemmatize_text)

In [ ]:
# Creating text variable for ham

df_ham = df[df.Category == 'ham']

text_ham = " ".join(text for text in df_ham['Message'])

In [ ]:
# Creating text variable for spam

df_spam = df[df.Category == 'spam']

text_spam = " ".join(text for text in df_spam['Message'])

In [ ]:
# Creating wordcloud for ham

ham_cloud = WordCloud(collocations = False, background_color = 'white').generate(text_ham)
plt.imshow(ham_cloud, interpolation='bilinear')
plt.axis("off")
plt.show()

<b> u, go, know, come, good, got, ok, time, ur, need, call etc. are some of the most common words in ham messages. </b> 

In [ ]:
# Creating wordcloud for spam

spam_cloud = WordCloud(collocations = False, background_color = 'white').generate(text_spam)
plt.imshow(spam_cloud, interpolation='bilinear')
plt.axis("off")
plt.show()

<b> call, free, now, txt, reply, text, claim, prize, mobile, new, service, stop, etc. are some of the most common words in spam messages. </b> 

## Model

In [ ]:
# Creating a Bag of Words model

cv = CountVectorizer()
X = cv.fit_transform(df['Message']).toarray()

In [ ]:
X

In [ ]:
# Encoding dependent variable

le = LabelEncoder()
df["Category"] = le.fit_transform(df["Category"])    # ham->0 and spam->1

In [ ]:
y = df["Category"]

In [ ]:
# Splitting the dataset into train and test 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)

In [ ]:
# Training the model using Naive Bayes classifier

nb = MultinomialNB().fit(X_train, y_train)

In [ ]:
print("Score of train data:", nb.score(X_train, y_train))
print("Score of test data:", nb.score(X_test, y_test))

<b> The model performs well on train as well as test data </b>

In [ ]:
y_pred = nb.predict(X_test)
y_pred

In [ ]:
# F1 score and accuracy

f1_score = f1_score(y_test, y_pred, average='weighted')
accuracy = accuracy_score(y_test, y_pred)

print("F1 Score:", f1_score)
print("Accuracy Score:", accuracy)

In [ ]:
# Classification Report

print(classification_report(y_test, y_pred))

In [ ]:
cm = confusion_matrix(y_test, y_pred)

group_names = ['True Neg', 'False Pos', 'False Neg', 'True Pos']
group_counts = ["{0:0.0f}".format(value) for value in
                cm.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in
                     cm.flatten()/np.sum(cm)]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cm, annot=labels, fmt='', cmap='Dark2')